In [ ]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical
from collections import deque
import matplotlib.pyplot as plt

# **Exercice 2 : Reinforcement learning dans une grille 5×5**

## **Description**
Un agent se déplace dans une grille de 5×5 cases en partant de la position (0,0) et doit atteindre un objectif situé à (4,4). À chaque étape, l'agent peut choisir l'une des quatre actions possibles :


1.  Haut (U).
2.  Bas (D).
3.  Gauche (L).
4.  Droite (R)

L'agent reçoit une récompense de +1 lorsqu'il atteint la cible et une récompense de -1 à chaque autre étape. S'il tente de sortir des limites de la grille, il reste sur place. L'objectif est d'entraîner un agent utilisant l'algorithme REINFORCE pour apprendre la meilleure politique de déplacement vers la cible en minimisant le nombre d'étapes.

# **Questions**

### **Compréhension de l’environnement**
1.   Quels sont les états possibles ($s_{t}$) de l’agent dans cet environnement ?
2.   Quelles sont les actions ($a_{t}$) que l’agent peut effectuer et comment influencent-elles son état ?
3.   Comment la fonction de récompense ($R_{t}$) est-elle définie et quel est son impact sur l’apprentissage ?

### **Implémentation avec REINFORCE**
1.   Comment représenter l’agent sous forme d’un réseau de neurones pour approximer la politique ?
2.   Quel est le rôle de la politique stochastique dans l’algorithme REINFORCE
3.   Comment la mise à jour des poids du réseau de neurones est-elle effectuée à partir des épisodes joués ?

### **Expérimentation et analyse**
1.   Après l'entraînement, comment pouvez-vous évaluer si l'agent a bien appris à atteindre la cible efficacement ?
2.   Que se passe-t-il si la grille devient plus grande (par exemple, 10×10) ? Quels ajustements seraient nécessaires dans l’apprentissage ?

# Compréhension de l'environnement

## 1. États possibles
Les états possibles ($s_t$) de l'agent sont toutes les positions qu'il peut occuper dans la grille 5×5. Chaque état est une paire de coordonnées (x, y) où x et y sont des entiers entre 0 et 4 inclus. Il y a donc 25 états possibles au total, allant de (0,0) à (4,4).

## 2. Actions possibles
L'agent peut effectuer quatre actions ($a_t$) :
- Haut (U) : déplace l'agent d'une case vers le haut (y+1)
- Bas (D) : déplace l'agent d'une case vers le bas (y-1)
- Gauche (L) : déplace l'agent d'une case vers la gauche (x-1)
- Droite (R) : déplace l'agent d'une case vers la droite (x+1)

Ces actions modifient l'état de l'agent en changeant ses coordonnées. Si l'action amène l'agent à sortir des limites de la grille (x < 0, y < 0, x > 4 ou y > 4), l'agent reste dans son état actuel.

## 3. Fonction de récompense
La fonction de récompense ($R_t$) est définie comme suit :
- +1 si l'agent atteint la cible (4,4)
- -1 pour chaque autre déplacement

Cette fonction de récompense encourage l'agent à atteindre la cible le plus rapidement possible, car chaque action qui ne mène pas directement à la cible est pénalisée. L'impact sur l'apprentissage est que l'agent va chercher à maximiser sa récompense cumulée en trouvant le chemin le plus court vers la cible.



# Implémentation avec REINFORCE

## 1. Représentation de l'agent sous forme de réseau de neurones

Dans cette implémentation, l'agent est représenté par un réseau de neurones qui approxime la politique. La structure est la suivante :
- Une couche d'entrée de dimension 2 (correspondant aux coordonnées x et y de l'agent)
- Une couche cachée de dimension 128 avec activation ReLU
- Une couche de sortie de dimension 4 (correspondant aux 4 actions possibles) avec activation softmax

La sortie du réseau est une distribution de probabilité sur les actions possibles, ce qui permet à l'agent de prendre des décisions stochastiques.

## 2. Rôle de la politique stochastique dans REINFORCE

La politique stochastique joue un rôle crucial dans l'algorithme REINFORCE pour plusieurs raisons :

1. **Exploration de l'environnement** : En choisissant les actions avec une certaine probabilité plutôt que de manière déterministe, l'agent explore différentes trajectoires dans l'environnement, ce qui lui permet de découvrir de meilleures stratégies.

2. **Échantillonnage des trajectoires** : REINFORCE est un algorithme de type Monte-Carlo qui nécessite d'échantillonner des trajectoires complètes. La stochasticité assure une diversité de trajectoires pour l'apprentissage.

3. **Calcul du gradient de la politique** : Dans REINFORCE, le gradient de la politique est estimé à partir des trajectoires échantillonnées. La formule mathématique du gradient implique les probabilités des actions, ce qui nécessite une politique stochastique.

Dans l'implémentation, la politique stochastique est obtenue en appliquant une fonction softmax à la sortie du réseau de neurones, ce qui donne une distribution de probabilité sur les actions. L'agent sélectionne ensuite une action en échantillonnant à partir de cette distribution.

## 3. Mise à jour des poids du réseau

La mise à jour des poids du réseau dans REINFORCE se fait selon les étapes suivantes :

1. Collecter une trajectoire complète (un épisode) en suivant la politique actuelle
2. Pour chaque étape t de la trajectoire, calculer le rendement G_t (somme des récompenses futures escomptées)
3. Mettre à jour les poids du réseau en utilisant le gradient de la politique pondéré par le rendement

Mathématiquement, la règle de mise à jour est :
θ = θ + α ∇_θ log π_θ(a_t|s_t) G_t

Où :
- θ représente les paramètres du réseau
- α est le taux d'apprentissage
- ∇_θ log π_θ(a_t|s_t) est le gradient du logarithme de la probabilité de l'action a_t dans l'état s_t
- G_t est le rendement à partir de l'étape t

Dans l'implémentation, cette mise à jour est effectuée dans la méthode `update_policy()` de la classe `ReinforceAgent`. Les rendements sont calculés, puis normalisés pour réduire la variance. Ensuite, la perte de la politique est calculée comme la somme des produits négatifs des logarithmes des probabilités des actions et des rendements correspondants. Enfin, la rétropropagation est utilisée pour mettre à jour les poids du réseau.



In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical

# Définition de l'environnement
class GridEnvironment:
    def __init__(self, grid_size=5):
        self.grid_size = grid_size
        self.reset()

    def reset(self):
        self.position = (0, 0)  # Position initiale
        self.target = (self.grid_size-1, self.grid_size-1)  # Position cible
        self.done = False
        return self._get_state()

    def _get_state(self):
        # Convertir la position en un état unique
        x, y = self.position
        return np.array([x, y])

    def step(self, action):
        # Définition des déplacements
        moves = {
            0: (0, 1),   # Haut (U)
            1: (0, -1),  # Bas (D)
            2: (-1, 0),  # Gauche (L)
            3: (1, 0)    # Droite (R)
        }

        x, y = self.position
        dx, dy = moves[action]
        new_x, new_y = x + dx, y + dy

        # Vérifier si la nouvelle position est dans les limites
        if 0 <= new_x < self.grid_size and 0 <= new_y < self.grid_size:
            self.position = (new_x, new_y)

        # Vérifier si l'agent a atteint la cible
        self.done = self.position == self.target

        # Définir la récompense
        if self.done:
            reward = 1.0
        else:
            reward = -1.0

        return self._get_state(), reward, self.done

# Réseau de neurones pour la politique
class PolicyNetwork(nn.Module):
    def __init__(self, state_dim, action_dim, hidden_dim=128):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(state_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=-1)

# Agent REINFORCE
class ReinforceAgent:
    def __init__(self, state_dim, action_dim, lr=0.001, gamma=0.99):
        self.policy = PolicyNetwork(state_dim, action_dim)
        self.optimizer = optim.Adam(self.policy.parameters(), lr=lr)
        self.gamma = gamma

        self.saved_log_probs = []
        self.rewards = []

    def select_action(self, state):
        state = torch.FloatTensor(state)
        probs = self.policy(state)
        m = Categorical(probs)
        action = m.sample()
        self.saved_log_probs.append(m.log_prob(action))
        return action.item()

    def update_policy(self):
        R = 0
        policy_loss = []
        returns = []

        # Calculer les rendements (retours) pour chaque pas de temps
        for r in self.rewards[::-1]:
            R = r + self.gamma * R
            returns.insert(0, R)

        returns = torch.tensor(returns)

        # Normaliser les rendements pour réduire la variance
        if len(returns) > 1:
            returns = (returns - returns.mean()) / (returns.std() + 1e-9)

        # Calculer la perte de la politique
        for log_prob, R in zip(self.saved_log_probs, returns):
            policy_loss.append(-log_prob * R)

        # Mettre à jour les poids du réseau
        if policy_loss:  # Vérifier que la liste n'est pas vide
            self.optimizer.zero_grad()
            # Utiliser stack au lieu de cat pour les tenseurs de dimension 0
            policy_loss = torch.stack(policy_loss).sum()
            policy_loss.backward()
            self.optimizer.step()

        # Réinitialiser les mémoires
        self.saved_log_probs = []
        self.rewards = []

# Fonction d'entraînement
def train(env, agent, max_episodes=1000, max_steps=100):
    episode_rewards = []

    for episode in range(max_episodes):
        state = env.reset()
        episode_reward = 0

        for step in range(max_steps):
            action = agent.select_action(state)
            next_state, reward, done = env.step(action)

            agent.rewards.append(reward)
            episode_reward += reward
            state = next_state

            if done:
                break

        agent.update_policy()
        episode_rewards.append(episode_reward)

        if episode % 100 == 0:
            print(f"Épisode {episode}, Récompense moyenne: {np.mean(episode_rewards[-100:])}")

    return episode_rewards

# Fonction d'évaluation
def evaluate(env, agent, num_episodes=10):
    total_steps = 0
    success = 0

    for _ in range(num_episodes):
        state = env.reset()
        done = False
        steps = 0

        while not done and steps < 100:
            # Utiliser la politique apprise de manière déterministe (prendre l'action la plus probable)
            state_tensor = torch.FloatTensor(state)
            with torch.no_grad():
                probs = agent.policy(state_tensor)
            action = probs.argmax().item()

            next_state, _, done = env.step(action)
            state = next_state
            steps += 1

        if done:
            success += 1
            total_steps += steps

    avg_steps = total_steps / success if success > 0 else float('inf')
    success_rate = success / num_episodes

    return avg_steps, success_rate

# Programme principal
if __name__ == "__main__":
    env = GridEnvironment(grid_size=5)
    state_dim = 2  # x, y
    action_dim = 4  # U, D, L, R

    agent = ReinforceAgent(state_dim, action_dim, lr=0.01, gamma=0.99)

    # Entraînement
    rewards = train(env, agent, max_episodes=2000, max_steps=100)

    # Évaluation
    avg_steps, success_rate = evaluate(env, agent, num_episodes=100)
    print(f"Taux de réussite: {success_rate:.2f}")
    print(f"Nombre moyen d'étapes pour atteindre la cible: {avg_steps:.2f}")

    # Visualiser la politique apprise
    grid_size = 5
    policy_grid = np.zeros((grid_size, grid_size), dtype=int)

    for y in range(grid_size):
        for x in range(grid_size):
            state = np.array([x, y])
            state_tensor = torch.FloatTensor(state)
            with torch.no_grad():
                probs = agent.policy(state_tensor)
            action = probs.argmax().item()
            policy_grid[grid_size-1-y, x] = action

    # Afficher la politique
    action_symbols = ['↑', '↓', '←', '→']
    for row in policy_grid:
        print(' '.join([action_symbols[a] for a in row]))

Épisode 0, Récompense moyenne: -100.0
Épisode 100, Récompense moyenne: -38.57
Épisode 200, Récompense moyenne: -6.52
Épisode 300, Récompense moyenne: -6.76
Épisode 400, Récompense moyenne: -7.02
Épisode 500, Récompense moyenne: -6.23
Épisode 600, Récompense moyenne: -6.06
Épisode 700, Récompense moyenne: -6.14
Épisode 800, Récompense moyenne: -6.05
Épisode 900, Récompense moyenne: -6.47
Épisode 1000, Récompense moyenne: -6.07
Épisode 1100, Récompense moyenne: -6.0
Épisode 1200, Récompense moyenne: -6.52
Épisode 1300, Récompense moyenne: -6.03
Épisode 1400, Récompense moyenne: -6.05
Épisode 1500, Récompense moyenne: -6.18
Épisode 1600, Récompense moyenne: -6.02
Épisode 1700, Récompense moyenne: -6.02
Épisode 1800, Récompense moyenne: -6.02
Épisode 1900, Récompense moyenne: -6.06
Taux de réussite: 1.00
Nombre moyen d'étapes pour atteindre la cible: 8.00
→ → → → ↑
→ → → ↑ ↑
→ → ↑ ↑ ↑
→ → ↑ ↑ ↑
→ ↑ ↑ ↑ ↑


# Expérimentation et analyse

## 1. Évaluation de l'apprentissage

Pour évaluer si l'agent a bien appris à atteindre la cible efficacement, on peut utiliser plusieurs métriques :

1. **Taux de réussite** : Proportion d'épisodes où l'agent atteint la cible dans une limite de temps/étapes donnée.
2. **Nombre moyen d'étapes** : Nombre moyen d'étapes nécessaires pour atteindre la cible. Dans notre cas, le chemin optimal nécessite 8 étapes (4 pas vers la droite et 4 pas vers le haut), donc un agent efficace devrait s'approcher de cette valeur.
3. **Récompense cumulée** : La somme des récompenses obtenues pendant un épisode. Pour le chemin optimal, cette valeur serait de -7+1 = -6 (7 étapes à -1 et l'arrivée à +1).
4. **Visualisation de la politique** : On peut représenter graphiquement la politique apprise en affichant l'action la plus probable pour chaque état.

Dans l'implémentation, la fonction `evaluate()` calcule le taux de réussite et le nombre moyen d'étapes, tandis que la dernière partie du code visualise la politique apprise sous forme de flèches dans une grille.

Un agent qui a bien appris aura un taux de réussite proche de 100%, un nombre moyen d'étapes proche de 8, et une politique qui montre clairement un chemin direct vers la cible.

## 2. Adaptation à une grille plus grande

Si la grille devient plus grande (par exemple, 10×10), plusieurs ajustements seraient nécessaires :

1. **Architecture du réseau** : Une grille plus grande peut nécessiter un réseau plus complexe, avec plus de neurones dans la couche cachée ou des couches supplémentaires pour capturer la complexité accrue de l'environnement.

2. **Paramètres d'apprentissage** :
   - Augmenter le nombre d'épisodes d'entraînement pour permettre une exploration plus complète
   - Ajuster le taux d'apprentissage, potentiellement le réduire pour une convergence plus stable
   - Modifier le facteur d'escompte γ pour équilibrer les récompenses à court et à long terme

3. **Exploration vs exploitation** : Dans un espace d'états plus grand, l'exploration devient plus critique. On pourrait :
   - Ajouter un mécanisme d'exploration explicite (comme epsilon-greedy)
   - Implémenter une planification de la température pour le softmax
   - Utiliser des techniques d'initialisation des poids qui favorisent l'exploration initiale

4. **Réduction de la variance** : Dans des environnements plus grands, la variance des estimations de gradient peut augmenter, ce qui ralentit l'apprentissage. Des techniques comme :
   - L'ajout d'une référence (baseline) pour réduire la variance
   - L'utilisation d'avantages au lieu de rendements bruts
   - L'implémentation d'algorithmes plus avancés comme A2C ou PPO

5. **Représentation des états** : Pour une grande grille, une représentation plus efficace des états pourrait être bénéfique, comme l'utilisation de la distance relative à la cible plutôt que des coordonnées absolues.

En pratique, pour une grille 10×10, on pourrait commencer par doubler le nombre d'épisodes d'entraînement, augmenter la taille de la couche cachée à 256 ou 512 neurones, et réduire légèrement le taux d'apprentissage pour assurer une convergence plus stable.